In [1]:
filename =  'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [3]:
import wget

data = wget.download(filename)
print(data)

 79% [...........................................................               ] 1179648 / 1475504

100% [..........................................................................] 1475504 / 1475504data (9).csv


In [4]:
data = pd.read_csv('data.csv')
data.head()

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350
3,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Coupe,28,18,3916,29450
4,BMW,1 Series,2011,premium unleaded (required),230.0,6.0,MANUAL,rear wheel drive,2.0,Luxury,Compact,Convertible,28,18,3916,34500


In [5]:
data.columns

Index(['Make', 'Model', 'Year', 'Engine Fuel Type', 'Engine HP',
       'Engine Cylinders', 'Transmission Type', 'Driven_Wheels',
       'Number of Doors', 'Market Category', 'Vehicle Size', 'Vehicle Style',
       'highway MPG', 'city mpg', 'Popularity', 'MSRP'],
      dtype='object')

In [6]:
data.columns = data.columns.str.replace(' ', '_').str.lower()

categorical_columns = list(data.dtypes[data.dtypes == 'object'].index)

for c in categorical_columns:
    data[c] = data[c].str.lower().str.replace(' ', '_')

In [7]:
data.columns

Index(['make', 'model', 'year', 'engine_fuel_type', 'engine_hp',
       'engine_cylinders', 'transmission_type', 'driven_wheels',
       'number_of_doors', 'market_category', 'vehicle_size', 'vehicle_style',
       'highway_mpg', 'city_mpg', 'popularity', 'msrp'],
      dtype='object')

In [8]:
df = data[['make', 'model', 'year', 'engine_hp', 'engine_cylinders', 'transmission_type', 
      'vehicle_style','highway_mpg', 'msrp']]

In [9]:
df.isnull().sum()

make                  0
model                 0
year                  0
engine_hp            69
engine_cylinders     30
transmission_type     0
vehicle_style         0
highway_mpg           0
msrp                  0
dtype: int64

In [10]:
def missing_num(df, fill):
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(fill)
    return df

In [11]:
zero = 0
df_no_missing = missing_num(df.copy(), zero)
df_no_missing.isnull().sum()

make                 0
model                0
year                 0
engine_hp            0
engine_cylinders     0
transmission_type    0
vehicle_style        0
highway_mpg          0
msrp                 0
dtype: int64

In [12]:
df_no_missing['price'] = df_no_missing.msrp
del df_no_missing['msrp']

In [13]:
df_no_missing

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,price
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,46135
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,40650
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,36350
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,29450
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,34500
...,...,...,...,...,...,...,...,...,...
11909,acura,zdx,2012,300.0,6.0,automatic,4dr_hatchback,23,46120
11910,acura,zdx,2012,300.0,6.0,automatic,4dr_hatchback,23,56670
11911,acura,zdx,2012,300.0,6.0,automatic,4dr_hatchback,23,50620
11912,acura,zdx,2013,300.0,6.0,automatic,4dr_hatchback,23,50920


In [14]:
df_no_missing['transmission_type'].mode()

0    automatic
dtype: object

In [15]:
df_no_missing.transmission_type.value_counts()

automatic           8266
manual              2935
automated_manual     626
direct_drive          68
unknown               19
Name: transmission_type, dtype: int64

In [16]:
df_no_missing.dtypes

make                  object
model                 object
year                   int64
engine_hp            float64
engine_cylinders     float64
transmission_type     object
vehicle_style         object
highway_mpg            int64
price                  int64
dtype: object

In [17]:
numerical = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'year']

In [18]:
df_no_missing[numerical].corr()

,engine_hp,engine_cylinders,highway_mpg,year
engine_hp,1.000000,0.774851,-0.415707,0.338714
engine_cylinders,0.774851,1.000000,-0.614541,-0.040708
highway_mpg,-0.415707,-0.614541,1.000000,0.258240
year,0.338714,-0.040708,0.258240,1.000000


In [19]:
def price_binary(df, price): 
    price_mean = df['price'].mean()
    df['above_average_' + price] = (df[price] > price_mean).astype(int)
    return df

In [20]:
df_new = price_binary(df_no_missing.copy(), 'price')
del df_new['price']

In [21]:
from sklearn.model_selection import train_test_split

df_fulltrain, df_test = train_test_split(df_new, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_fulltrain, test_size=0.25, random_state=42)

In [22]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [23]:
y_train = df_train.above_average_price.values
y_val = df_val.above_average_price.values
y_test = df_test.above_average_price.values

In [24]:
del df_train['above_average_price']
del df_val['above_average_price']
del df_test['above_average_price']

In [25]:
categorical = ['make', 'model', 'transmission_type', 'vehicle_style']

In [26]:
from sklearn.metrics import mutual_info_score

In [27]:
def mutual_info_above_avg_score(series):
    return mutual_info_score(series, df_fulltrain.above_average_price)

In [28]:
mi = df_fulltrain[categorical].apply(mutual_info_above_avg_score)
score = mi.sort_values(ascending=False)
round(score, 2)

model                0.46
make                 0.24
vehicle_style        0.08
transmission_type    0.02
dtype: float64

In [29]:
from sklearn.feature_extraction import DictVectorizer

In [30]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [31]:
dv = DictVectorizer(sparse=False)

In [32]:
X_train = dv.fit_transform(train_dicts)

In [33]:
dv.get_feature_names_out()

array(['engine_cylinders', 'engine_hp', 'highway_mpg', 'make=acura',
       'make=alfa_romeo', 'make=aston_martin', 'make=audi',
       'make=bentley', 'make=bmw', 'make=bugatti', 'make=buick',
       'make=cadillac', 'make=chevrolet', 'make=chrysler', 'make=dodge',
       'make=ferrari', 'make=fiat', 'make=ford', 'make=genesis',
       'make=gmc', 'make=honda', 'make=hummer', 'make=hyundai',
       'make=infiniti', 'make=kia', 'make=lamborghini', 'make=land_rover',
       'make=lexus', 'make=lincoln', 'make=lotus', 'make=maserati',
       'make=maybach', 'make=mazda', 'make=mclaren', 'make=mercedes-benz',
       'make=mitsubishi', 'make=nissan', 'make=oldsmobile',
       'make=plymouth', 'make=pontiac', 'make=porsche',
       'make=rolls-royce', 'make=saab', 'make=scion', 'make=spyker',
       'make=subaru', 'make=suzuki', 'make=tesla', 'make=toyota',
       'make=volkswagen', 'make=volvo', 'model=100', 'model=124_spider',
       'model=190-class', 'model=1_series', 'model=2', 'model=

In [34]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [35]:
X_val = dv.transform(val_dicts)

In [36]:
from sklearn.linear_model import LogisticRegression

In [37]:
model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)

In [38]:
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [39]:
model.predict_proba(X_train)[:, 1]

array([3.84167872e-02, 9.81481058e-02, 9.99642644e-01, ...,
       4.89199860e-02, 3.82507375e-04, 5.69784875e-01])

In [40]:
y_pred = model.predict_proba(X_val)[:, 1]

In [41]:
decision = (y_pred >= 0.5)

In [42]:
decision = decision.astype(int)
decision

array([0, 1, 0, ..., 0, 1, 1])

In [43]:
original_score = round((y_val == decision).mean(), 2)
original_score

0.93

In [44]:
data = categorical + numerical
data

['make',
 'model',
 'transmission_type',
 'vehicle_style',
 'engine_hp',
 'engine_cylinders',
 'highway_mpg',
 'year']

In [45]:
original_score 
def feat_elimination(df):
    
    for col in df:
        df_sub = df.copy()
        df_sub.remove(col)
        train_dicts = df_train[df_sub].to_dict(orient='records')

        dv = DictVectorizer(sparse=False)

        X_train = dv.fit_transform(train_dicts)
        val_dicts = df_val[df_sub].to_dict(orient='records')
        X_val = dv.transform(val_dicts)

        from sklearn.linear_model import LogisticRegression

        model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
        model.fit(X_train, y_train)

       
        y_pred = model.predict_proba(X_val)[:, 1]

        decision = (y_pred >= 0.5)
        decision = decision.astype(int)
        score = round((y_val == decision).mean(), 2)

        print(col, original_score - score, score)

In [46]:
feat_elimination(data)

make -0.009999999999999898 0.94
model 0.010000000000000009 0.92
transmission_type -0.019999999999999907 0.95
vehicle_style -0.009999999999999898 0.94
engine_hp 0.010000000000000009 0.92
engine_cylinders -0.009999999999999898 0.94
highway_mpg 0.010000000000000009 0.92
year -0.019999999999999907 0.95


In [47]:
df_fulltrain, df_test = train_test_split(df_no_missing, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_fulltrain, test_size=0.25, random_state=42)

In [48]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [49]:
y_train = np.log1p(df_train['price'].values)
y_val = np.log1p(df_val['price'].values)
y_test = np.log1p(df_test['price'].values)

In [50]:
del df_train['price']
del df_val['price']
del df_test['price']

In [52]:
train_dicts = df_train[categorical + numerical].to_dict(orient='records')

In [53]:
dv = DictVectorizer(sparse=False)

In [54]:
X_train = dv.fit_transform(train_dicts)

In [55]:
val_dicts = df_val[categorical + numerical].to_dict(orient='records')

In [56]:
X_val = dv.transform(val_dicts)

In [57]:
from sklearn.linear_model import Ridge

In [60]:
def rmse(y, y_pred):
    se = (y - y_pred) ** 2
    mse = se.mean()
    return np.sqrt(mse)
#rmse(y_train, y_pred)

In [61]:
for value in [0, 0.01, 0.1, 1, 10]:
    model = Ridge(alpha=value, solver='sag', random_state=42)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    score = rmse(y_val, y_pred)
    
    print(value, round(score, 3))

0 0.497
0.01 0.497
0.1 0.497
1 0.497
10 0.497
